# SVM  

This will start with using SKLearn, but might change to Tensorflow

For now, only one stock will be used, so for now my file that takes in data will find that one stock, and it will be one from the sp500

In [608]:
import pandas as pd
import numpy as np

def stock_csv_parser(stock):

    fileName = "stock_market_data/sp500/csv/" + stock + ".csv"
    df = pd.read_csv(fileName)

    return df

symbol_df = stock_csv_parser("GE")

# I want to get the percentage change, so if I wanted to make a threshold such as 2%
# I will first drop any open if it is zero just in case to prevent division by zero
symbol_df = symbol_df[symbol_df['Open'] != 0]
y = symbol_df['Close'] / symbol_df['Open']

y = [1 if each > 1.01 else 0 for each in y]

# print(y)
print("data points:", str(len(y)), "\t Results as success:", np.count_nonzero(y))
symbol_df

data points: 13356 	 Results as success: 2607


,Date,Low,Open,Volume,High,Close,Adjusted Close
0,02-01-1970,6.129808,6.209936,289536,6.219952,6.139824,1.267613
1,05-01-1970,6.059696,6.139824,529152,6.169872,6.109776,1.261409
2,06-01-1970,5.899439,6.099760,443040,6.099760,5.929487,1.224188
3,07-01-1970,5.949519,5.949519,575328,6.039663,5.959535,1.230390
4,08-01-1970,5.979567,5.979567,1737216,6.069712,6.009615,1.240731
...,...,...,...,...,...,...,...
13351,06-12-2022,84.470001,86.610001,5883900,87.099998,85.279999,85.279999
13352,07-12-2022,84.820000,85.000000,4860600,86.080002,85.000000,85.000000
13353,08-12-2022,82.690002,85.599998,6438400,86.190002,83.629997,83.629997
13354,09-12-2022,81.330002,83.620003,5539000,83.800003,81.449997,81.449997


Now that we have one of the stock in, we will try to moddify the data, then train it with svm for now
I will come back later to get it to do more than just one stock, this is for quicker testing for now

In [609]:
from sklearn.preprocessing import StandardScaler

try:
    symbol_df = symbol_df.drop(['Date'], axis = 1)
except:
    print("Column missing")

print(symbol_df)

scaler = StandardScaler()
X = scaler.fit_transform(symbol_df)

print(X)


             Low       Open   Volume       High      Close  Adjusted Close
0       6.129808   6.209936   289536   6.219952   6.139824        1.267613
1       6.059696   6.139824   529152   6.169872   6.109776        1.261409
2       5.899439   6.099760   443040   6.099760   5.929487        1.224188
3       5.949519   5.949519   575328   6.039663   5.959535        1.230390
4       5.979567   5.979567  1737216   6.069712   6.009615        1.240731
...          ...        ...      ...        ...        ...             ...
13351  84.470001  86.610001  5883900  87.099998  85.279999       85.279999
13352  84.820000  85.000000  4860600  86.080002  85.000000       85.000000
13353  82.690002  85.599998  6438400  86.190002  83.629997       83.629997
13354  81.330002  83.620003  5539000  83.800003  81.449997       81.449997
13355  81.029999  81.720001  1109941  81.940002  81.824997       81.824997

[13356 rows x 6 columns]
[[-0.98525854 -0.98501805 -0.75006802 -0.98525494 -0.98544583 -1.00434018]

I am unsure if I should do PCA first or append 10 days to each other then do it
I suspect I will do PCA after since each day will be highly correlated to the previous day

In [610]:
def X_day_appender(day_info, day_ct=10):
    result = []
    for i in range(len(day_info) - (day_ct)):
        result.append(day_info[i:i+day_ct, :].flatten())
    return result

day_ct = 10
X = X_day_appender(X, day_ct)

y = y[day_ct:]

I want to now use PCA because this data will all naturally be correlated

In [611]:
from sklearn.decomposition import PCA

pca = PCA(n_components=7)
X_PCA = pca.fit_transform(X)

print(X_PCA[0], len(X_PCA), len(y))


[-7.1937395  -1.40416431  0.02633708  0.09015554  0.22120365 -0.06508217
 -0.07190814] 13346 13346


Now that I have done PCA, I want to split my data so that future data is not trained on

In [612]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# This will split up my data, while keeping it in order
X_train, X_test, y_train, y_test = train_test_split(X_PCA, y, test_size=0.2, shuffle=False)

# I am removing some that are close or are in the train set at least part of the way
X_test = X_test[day_ct+5:]
y_test = y_test[day_ct+5:]

# I am then shuffleing the data, thought that might have only been NN that needed this step, I am unsure
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)



Now that I have done quite a bit to the data, I will train it using SVM
I want to see if the results can have a high percision
I think percision is what I want the most because I do not want to have bad days be prediceted as good days

I will first be trying svc, because I want a classification
I might move to one class svm because there is a great deal of the results not meeting the threshold

I am using svc and not linear svc, because the documentation suggested linear svc if you have more than 10s of thousands of samples
Which when only doing one stock with this data set, there is not too many samples

In [613]:
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


Now that it is train, I want to see what I can get out of the results

In [614]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy: ", str(accuracy),  "\tPrecision: ", str(precision), "\tRecall: ", str(recall), "\tNumber of predicted days: ", str(np.count_nonzero(y_pred)))

Accuracy:  0.8090395480225989 	Precision:  0.6428571428571429 	Recall:  0.01761252446183953 	Number of predicted days:  14


So far the results are not great, I will continute to work to see if I can get any sort of result from this
I have tried a few stocks and General Electric is returning the best results so far

I plan to also add in a way to see the change in value if when predicted to be a good day, the stock is bought at open and sold at close
I plan to also try to test all sp500 stocks at once to see which ones give the best results